In [1]:
import re
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


import torch
from collections import Counter
from torch.utils.data import DataLoader, TensorDataset

from konlpy.tag import Okt, Komoran, Kkma


In [2]:
# num_samples = 33000
tokenizer = Kkma()

In [3]:
rawDF = pd.read_csv('../pronunciation_standard.txt', sep='\t')
rawDF.head(5)

,pronunciation,standard
0,오랫동안 한 동네에서 살았던 할무닌데 도라가주씨 매매 아프네요,오랫동안 한 동네에서 살았던 할머니인데 돌아가겨서 마음이 아프네요
1,혈압약은 시간을 맞춰 챙겨 드셔야지 안 그러면 효과가 없습니다,혈압약은 시간을 맞춰 챙겨 드셔야지 안 그러면 효과가 없습니다
2,집에 돌아와 보이꺼네 문이 열려 있고 뼈다지가 열어둔 돈 전부 없어지던 어이떼,집에 돌아와 보니까 문이 열려 있고 서랍이 열어둔 돈 전부 없어지던 어이떼
3,아들 오늘 중요한 시험 보니까네 이 생서 하고 사가꼬 묵고 힘내서 시험 잘 바래이,아들 오늘 중요한 시험 보니까 이 생 엿 하고 사서 먹고 힘내서 시험 잘 봐
4,옛날부터 조상꿈이나 돼지꿈 꾸만 집에 돈 많이 들어온다고 좋아 해찌로,옛날부터 조상꿈이나 돼지꿈 꾸면 집에 돈 많이 들어온다고 좋아 했죠


In [4]:
max_src_len = max( [len(line) for line in rawDF.pronunciation])
print(max_src_len)

70


In [5]:
def cleanTxt(sent):
	# 한글 공백 제외 삭제
	sent = re.sub(r"[^가-힣\s]", "", sent)
	return sent

In [6]:
def load_preprocessed_data(tokenizer):
    encoder_input, decoder_input, decoder_target = [], [], []

    with open("../pronunciation_standard.txt", "r") as lines:
        # print(lines)											# <_io.TextIOWrapper name='../dialect_standard.txt' mode='r' encoding='UTF-8'>
        for i, line in enumerate(lines):
            
            # source 데이터와 target 데이터 분리
            if not i: continue									# header 라인 패스
            elif len(line.strip().split('\t'))==2:
                
                # print(line.strip().split('\t'))					# ['혈압약은 시간을', '혈압약은 시간을']
                
				# 표준어 	방언
                src_line, tar_line = line.strip().split('\t')[1], line.strip().split('\t')[0]
                # print(src_line)
                # print(tar_line)
	
				# source 데이터 전처리
                # print(cleanTxt(src_line))
                src_line = [w for w in tokenizer.morphs(cleanTxt(src_line))]
                print(src_line)
                # break

				# target 데이터 전처리
                tar_line = tokenizer.morphs(cleanTxt(tar_line))
                tar_line_in = [w for w in (["<sos>"] + tar_line)]
                tar_line_out = [w for w in (tar_line + ["<eos>"])]

                encoder_input.append(src_line)
                decoder_input.append(tar_line_in)
                decoder_target.append(tar_line_out)
                

            # if i == len(lines) - 1:
            #     break

    return encoder_input, decoder_input, decoder_target

In [77]:
# tmp = "올해는
#  배치 농사를 망우가 짐장을 할라믄 배치를 좀 사와가 담구야 데겠네"	"올해는
#  배추 농사를 망쳐서 김장을 하려면 배추를 좀 사와서 담궈야 되겠네"

In [7]:
sents_std_in, sents_di_in, sents_di_out = load_preprocessed_data(tokenizer)
print('인코더의 입력 :',sents_std_in[:5])
print('디코더의 입력 :',sents_di_in[:5])
print('디코더의 레이블 :',sents_di_out[:5])


['오랫동안', '한', '동네', '에서', '살', '았', '더', 'ㄴ', '할머니', '이', 'ㄴ데', '돌아가겨', '서', '마음', '이', '아프', '네요']
['혈압', '약', '은', '시간', '을', '맞추', '어', '챙기', '어', '드시', '어야지', '안', '그러', '면', '효과', '가', '없', '습니다']
['집', '에', '돌아오', '아', '보', '니까', '문', '이', '열리', '어', '있', '고', '서랍', '이', '열', '어', '두', 'ㄴ', '돈', '전부', '없', '어', '지', '던', '어이', '떼']
['아들', '오늘', '중요', '하', 'ㄴ', '시험', '보', '니까', '이', '생', '엿', '하', '고', '사서', '먹', '고', '힘', '내서', '시험', '잘', '보', '아']
['옛날', '부터', '조상', '꿈', '이나', '돼지', '꿈', '꾸', '면', '집', '에', '돈', '많이', '들어오', 'ㄴ다고', '좋', '아', '하', '었', '죠']
['겨울', '에', '먹', '을', '채소', '나', '과일', '같', '은', '것', '은', '어디', '에', '보관', '을', '하', '었', '습니까']
['친정', '동네', '하', '고', '이', '동네', '하', '고', '차례', '지내', 'ㄹ', '때', '상', '차리', '는', '것', '은', '뭐', '가', '다르', 'ㅂ니까']
['친정', '동네', '하', '고', '이', '동네', '하', '고', '차례', '지내', 'ㄹ', '때', '상', '차리', '는', '것', '은', '뭐', '가', '다르', 'ㅂ니까']
['촌구석', '이', '라', '젊', '은', '사람', '들', '은', '아예', '없', '고', '전부', '노인', '들', '만', '있', '으니까', '농사', '짓

In [8]:
sents_std_in[:3]

[['오랫동안',
  '한',
  '동네',
  '에서',
  '살',
  '았',
  '더',
  'ㄴ',
  '할머니',
  '이',
  'ㄴ데',
  '돌아가겨',
  '서',
  '마음',
  '이',
  '아프',
  '네요'],
 ['혈압',
  '약',
  '은',
  '시간',
  '을',
  '맞추',
  '어',
  '챙기',
  '어',
  '드시',
  '어야지',
  '안',
  '그러',
  '면',
  '효과',
  '가',
  '없',
  '습니다'],
 ['집',
  '에',
  '돌아오',
  '아',
  '보',
  '니까',
  '문',
  '이',
  '열리',
  '어',
  '있',
  '고',
  '서랍',
  '이',
  '열',
  '어',
  '두',
  'ㄴ',
  '돈',
  '전부',
  '없',
  '어',
  '지',
  '던',
  '어이',
  '떼']]

In [9]:
set(sents_std_in[0])

{'ㄴ',
 'ㄴ데',
 '네요',
 '더',
 '돌아가겨',
 '동네',
 '마음',
 '살',
 '서',
 '아프',
 '았',
 '에서',
 '오랫동안',
 '이',
 '한',
 '할머니'}

In [10]:
def build_vocab(sents):
  word_list = []

  for sent in sents:
      for word in sent:
        word_list.append(word)

  # 각 단어별 등장 빈도를 계산하여 등장 빈도가 높은 순서로 정렬
#   word_counts = Counter(word_list)
#   vocab = sorted(word_counts, key=word_counts.get, reverse=True)
  
  vocab = set(word_list)

  word_to_index = {}
  word_to_index['<PAD>'] = 0
  word_to_index['<UNK>'] = 1

  # 등장 빈도가 높은 단어일수록 낮은 정수를 부여
  for index, word in enumerate(vocab) :
    word_to_index[word] = index + 2

  return word_to_index


In [11]:
src_vocab = build_vocab(sents_std_in)
tar_vocab = build_vocab(sents_di_in + sents_di_out)

src_vocab_size = len(src_vocab)
tar_vocab_size = len(tar_vocab)
print("표준어 단어 집합의 크기 : {:d}, 방언 단어 집합의 크기 : {:d}".format(src_vocab_size, tar_vocab_size))

표준어 단어 집합의 크기 : 2718, 방언 단어 집합의 크기 : 5037


In [12]:
src_vocab

{'<PAD>': 0,
 '<UNK>': 1,
 '식': 2,
 '쫙': 3,
 '글': 4,
 '생선': 5,
 '초가': 6,
 '비빔밥': 7,
 '방법': 8,
 '북': 9,
 '콘트트': 10,
 '제대': 11,
 '차리': 12,
 '강가': 13,
 '내딛': 14,
 '가물': 15,
 '슬': 16,
 '어떠': 17,
 '배치': 18,
 '으나': 19,
 '지만': 20,
 'ㄴ지요': 21,
 '사내아이': 22,
 '친구': 23,
 '붙이': 24,
 '모숨': 25,
 '테': 26,
 '있느더이': 27,
 '연구': 28,
 '비난': 29,
 '싹': 30,
 '든지': 31,
 '말': 32,
 '었': 33,
 '공부': 34,
 '뺏': 35,
 '낮': 36,
 '허리': 37,
 '여기': 38,
 '에리베이터에': 39,
 '깻엿': 40,
 '엉키': 41,
 '구려': 42,
 '엘리버이터': 43,
 '날': 44,
 '휴식': 45,
 '보호': 46,
 '세트': 47,
 '살갗': 48,
 '비상구': 49,
 '켔': 50,
 '려고': 51,
 '땡': 52,
 '끼': 53,
 '잘리': 54,
 '중간': 55,
 '벌': 56,
 '아야지요': 57,
 '지정': 58,
 '영': 59,
 '인사': 60,
 '용': 61,
 '일드': 62,
 '씹': 63,
 '공이': 64,
 '갓난아이': 65,
 '짝': 66,
 '얼렁': 67,
 '무로': 68,
 '의': 69,
 '우르르': 70,
 '가보': 71,
 '망치': 72,
 '죽': 73,
 '무엇': 74,
 '하다': 75,
 '면': 76,
 '중': 77,
 '절': 78,
 '개': 79,
 '겨울철': 80,
 '뿌라': 81,
 '달라': 82,
 '깐에': 83,
 '정갈': 84,
 '된': 85,
 '등에': 86,
 '땔감': 87,
 '참고': 88,
 '메': 89,
 '가장': 90,
 '워지': 91,

In [13]:
index_to_src = {v: k for k, v in src_vocab.items()}
index_to_tar = {v: k for k, v in tar_vocab.items()}

def texts_to_sequences(sents, word_to_index):
  encoded_X_data = []
  for sent in sents:
    index_sequences = []
    for word in sent:
      try:
          index_sequences.append(word_to_index[word])
      except KeyError:
          index_sequences.append(word_to_index['<UNK>'])
    encoded_X_data.append(index_sequences)
  return encoded_X_data


In [14]:
encoder_input = texts_to_sequences(sents_std_in, src_vocab)
decoder_input = texts_to_sequences(sents_di_in, tar_vocab)
decoder_target = texts_to_sequences(sents_di_out, tar_vocab)


In [15]:
# 상위 5개의 샘플에 대해서 정수 인코딩 전, 후 문장 출력
# 인코더 입력이므로 <sos>나 <eos>가 없음
for i, (item1, item2) in zip(range(5), zip(sents_std_in, encoder_input)):
    print(f"Index: {i}, 정수 인코딩 전: {item1}, 정수 인코딩 후: {item2}")


Index: 0, 정수 인코딩 전: ['오랫동안', '한', '동네', '에서', '살', '았', '더', 'ㄴ', '할머니', '이', 'ㄴ데', '돌아가겨', '서', '마음', '이', '아프', '네요'], 정수 인코딩 후: [1310, 1340, 652, 998, 916, 507, 2351, 1977, 168, 2636, 1975, 303, 2303, 2107, 2636, 526, 1038]
Index: 1, 정수 인코딩 전: ['혈압', '약', '은', '시간', '을', '맞추', '어', '챙기', '어', '드시', '어야지', '안', '그러', '면', '효과', '가', '없', '습니다'], 정수 인코딩 후: [620, 1094, 618, 1734, 681, 1411, 2060, 2204, 2060, 992, 735, 905, 1567, 76, 1925, 2449, 2309, 2200]
Index: 2, 정수 인코딩 전: ['집', '에', '돌아오', '아', '보', '니까', '문', '이', '열리', '어', '있', '고', '서랍', '이', '열', '어', '두', 'ㄴ', '돈', '전부', '없', '어', '지', '던', '어이', '떼'], 정수 인코딩 후: [1798, 1279, 1347, 769, 407, 2426, 1610, 2636, 1455, 2060, 648, 1972, 2234, 2636, 2446, 2060, 2645, 1977, 1801, 1101, 2309, 2060, 2350, 1907, 604, 2009]
Index: 3, 정수 인코딩 전: ['아들', '오늘', '중요', '하', 'ㄴ', '시험', '보', '니까', '이', '생', '엿', '하', '고', '사서', '먹', '고', '힘', '내서', '시험', '잘', '보', '아'], 정수 인코딩 후: [2367, 1241, 2507, 2225, 1977, 401, 407, 2426, 2636, 920, 1112, 222

In [16]:
def pad_sequences(sentences, max_len=None):
    # 최대 길이 값이 주어지지 않을 경우 데이터 내 최대 길이로 패딩
    if max_len is None:
        max_len = max([len(sentence) for sentence in sentences])

    features = np.zeros((len(sentences), max_len), dtype=int)
    for index, sentence in enumerate(sentences):
        if len(sentence) != 0:
            features[index, :len(sentence)] = np.array(sentence)[:max_len]
    return features


In [17]:
encoder_input = pad_sequences(encoder_input)
decoder_input = pad_sequences(decoder_input)
decoder_target = pad_sequences(decoder_target)

In [18]:
print('인코더의 입력의 크기(shape) :',encoder_input.shape)
print('디코더의 입력의 크기(shape) :',decoder_input.shape)
print('디코더의 레이블의 크기(shape) :',decoder_target.shape)


인코더의 입력의 크기(shape) : (15296, 40)
디코더의 입력의 크기(shape) : (15296, 43)
디코더의 레이블의 크기(shape) : (15296, 43)


In [19]:
type( encoder_input )

numpy.ndarray

In [20]:
indices = np.arange(encoder_input.shape[0])
np.random.shuffle(indices)
print('랜덤 시퀀스 :',indices)


랜덤 시퀀스 : [ 6055  2355 14613 ...   429 10732 11099]


In [21]:
encoder_input = encoder_input[indices]
decoder_input = decoder_input[indices]
decoder_target = decoder_target[indices]


In [22]:
print([index_to_src[word] for word in encoder_input[12300]])
print([index_to_tar[word] for word in decoder_input[12300]])
print([index_to_tar[word] for word in decoder_target[12300]])


['방', '에', '가래떡', '하', '고', '술', '을', '차리', '어', '놓', '아', '쓰', '니까', '편하', '게', '드시', '면서', '말씀', '나누', '세요', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['<sos>', '방', '에', '가리', '떡', '하', '고', '술', '을', '차리', '어', '놓', '아', '쓰', '이', '꺼', '네', '편하', '게', '드시', '면서', '말씀', '나누', '시', '소', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']
['방', '에', '가리', '떡', '하', '고', '술', '을', '차리', '어', '놓', '아', '쓰', '이', '꺼', '네', '편하', '게', '드시', '면서', '말씀', '나누', '시', '소', '<eos>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>', '<PAD>']


In [23]:
n_of_train = int(15296*0.7) +1
n_of_val = int(15296*0.2)
n_of_test = int(15296*0.1)


print('훈련 데이터의 개수 :',n_of_train)
print('검증 데이터의 개수 :',n_of_val)
print('테스트 데이터의 개수 :',n_of_test)

print("총합",n_of_train+n_of_val+n_of_test)

훈련 데이터의 개수 : 10708
검증 데이터의 개수 : 3059
테스트 데이터의 개수 : 1529
총합 15296


In [24]:
encoder_input

array([[ 438, 1279, 2449, ...,    0,    0,    0],
       [2454,  681, 2116, ...,    0,    0,    0],
       [2551, 2636, 2412, ...,    0,    0,    0],
       ...,
       [2635, 2258, 2092, ...,    0,    0,    0],
       [2356, 1399, 1977, ...,    0,    0,    0],
       [1333, 1897,  905, ...,    0,    0,    0]])

In [25]:
encoder_input[n_of_train:n_of_train+n_of_val+1,:]

array([[ 305, 2356,  216, ...,    0,    0,    0],
       [1372, 1641, 2308, ...,    0,    0,    0],
       [ 996, 1279,  121, ...,    0,    0,    0],
       ...,
       [2134, 1279, 1589, ...,    0,    0,    0],
       [2356, 2357, 2362, ...,    0,    0,    0],
       [2260,  681, 2439, ...,    0,    0,    0]])

In [26]:
encoder_input_train = encoder_input[:n_of_train, :]
decoder_input_train = decoder_input[:n_of_train, :]
decoder_target_train = decoder_target[:n_of_train, :]

encoder_input_val = encoder_input[n_of_train:n_of_train+n_of_val+1, :]
decoder_input_val = decoder_input[n_of_train:n_of_train+n_of_val+1, :]
decoder_target_val = decoder_target[n_of_train:n_of_train+n_of_val+1, :]

encoder_input_test = encoder_input[-n_of_test:]
decoder_input_test = decoder_input[-n_of_test:]
decoder_target_test = decoder_target[-n_of_test:]

In [27]:
print('훈련 source 데이터의 크기 :',encoder_input_train.shape)
print('훈련 target 데이터의 크기 :',decoder_input_train.shape)
print('훈련 target 레이블의 크기 :',decoder_target_train.shape)
print()
print('검증 source 데이터의 크기 :',encoder_input_val.shape)
print('검증 target 데이터의 크기 :',decoder_input_val.shape)
print('검증 target 레이블의 크기 :',decoder_target_val.shape)
print()
print('테스트 source 데이터의 크기 :',encoder_input_test.shape)
print('테스트 target 데이터의 크기 :',decoder_input_test.shape)
print('테스트 target 레이블의 크기 :',decoder_target_test.shape)

훈련 source 데이터의 크기 : (10708, 40)
훈련 target 데이터의 크기 : (10708, 43)
훈련 target 레이블의 크기 : (10708, 43)

검증 source 데이터의 크기 : (3060, 40)
검증 target 데이터의 크기 : (3060, 43)
검증 target 레이블의 크기 : (3060, 43)

테스트 source 데이터의 크기 : (1529, 40)
테스트 target 데이터의 크기 : (1529, 43)
테스트 target 레이블의 크기 : (1529, 43)


#### 번역기 만들기

In [28]:
import torch
import torch.nn as nn
import torch.optim as optim

embedding_dim = 256
hidden_units = 256

class Encoder(nn.Module):
    def __init__(self, src_vocab_size, embedding_dim, hidden_units):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(src_vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_units, batch_first=True)

    def forward(self, x):
        # x.shape == (batch_size, seq_len, embedding_dim)
        x = self.embedding(x)
        # hidden.shape == (1, batch_size, hidden_units), cell.shape == (1, batch_size, hidden_units)
        _, (hidden, cell) = self.lstm(x)
        # 인코더의 출력은 hidden state, cell state
        return hidden, cell

class Decoder(nn.Module):
    def __init__(self, tar_vocab_size, embedding_dim, hidden_units):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(tar_vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_units, batch_first=True)
        self.fc = nn.Linear(hidden_units, tar_vocab_size)

    def forward(self, x, hidden, cell):

        # x.shape == (batch_size, seq_len, embedding_dim)
        x = self.embedding(x)

        # 디코더의 LSTM으로 인코더의 hidden state, cell state를 전달.
        # output.shape == (batch_size, seq_len, hidden_units)
        # hidden.shape == (1, batch_size, hidden_units)
        # cell.shape == (1, batch_size, hidden_units)
        output, (hidden, cell) = self.lstm(x, (hidden, cell))

        # output.shape: (batch_size, seq_len, tar_vocab_size)
        output = self.fc(output)

        # 디코더의 출력은 예측값, hidden state, cell state
        return output, hidden, cell

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super(Seq2Seq, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, trg):
        hidden, cell = self.encoder(src)

        # 훈련 중에는 디코더의 출력 중 오직 output만 사용한다.
        output, _, _ = self.decoder(trg, hidden, cell)
        return output




In [29]:
encoder = Encoder(src_vocab_size, embedding_dim, hidden_units)
decoder = Decoder(tar_vocab_size, embedding_dim, hidden_units)
model = Seq2Seq(encoder, decoder)

LR = 0.001
loss_function = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(model.parameters(), lr=LR)

In [30]:
print(model)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2718, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(5037, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=5037, bias=True)
  )
)


In [31]:
def evaluation(model, dataloader, loss_function, device):
    model.eval()
    total_loss = 0.0
    total_correct = 0
    total_count = 0

    with torch.no_grad():
        for encoder_inputs, decoder_inputs, decoder_targets in dataloader:
            encoder_inputs = encoder_inputs.to(device)
            decoder_inputs = decoder_inputs.to(device)
            decoder_targets = decoder_targets.to(device)

            # 순방향 전파
            # outputs.shape == (batch_size, seq_len, tar_vocab_size)
            outputs = model(encoder_inputs, decoder_inputs)

            # 손실 계산
            # outputs.view(-1, outputs.size(-1))의 shape는 (batch_size * seq_len, tar_vocab_size)
            # decoder_targets.view(-1)의 shape는 (batch_size * seq_len)
            loss = loss_function(outputs.view(-1, outputs.size(-1)), decoder_targets.view(-1))
            total_loss += loss.item()

            # 정확도 계산 (패딩 토큰 제외)
            mask = decoder_targets != 0
            total_correct += ((outputs.argmax(dim=-1) == decoder_targets) * mask).sum().item()
            total_count += mask.sum().item()

    return total_loss / len(dataloader), total_correct / total_count


In [32]:
encoder_input_train_tensor = torch.tensor(encoder_input_train, dtype=torch.long)
decoder_input_train_tensor = torch.tensor(decoder_input_train, dtype=torch.long)
decoder_target_train_tensor = torch.tensor(decoder_target_train, dtype=torch.long)

encoder_input_val_tensor = torch.tensor(encoder_input_val, dtype=torch.long)
decoder_input_val_tensor = torch.tensor(decoder_input_val, dtype=torch.long)
decoder_target_val_tensor = torch.tensor(decoder_target_val, dtype=torch.long)

# 데이터셋 및 데이터로더 생성
batch_size = 128

train_dataset = TensorDataset(encoder_input_train_tensor, decoder_input_train_tensor, decoder_target_train_tensor)
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

valid_dataset = TensorDataset(encoder_input_val_tensor, decoder_input_val_tensor, decoder_target_val_tensor)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False)


### 학습 설정

In [33]:
# 학습 설정
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(2718, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(5037, 256, padding_idx=0)
    (lstm): LSTM(256, 256, batch_first=True)
    (fc): Linear(in_features=256, out_features=5037, bias=True)
  )
)

In [62]:
# Training loop
best_val_loss = float('inf')

train_history = {'Loss':[], 'Acc':[]}
valid_history = {'Loss':[], 'Acc':[]}


for epoch in range(num_epochs):
    # 훈련 모드
    model.train()

    for encoder_inputs, decoder_inputs, decoder_targets in train_dataloader:
        encoder_inputs = encoder_inputs.to(device)
        decoder_inputs = decoder_inputs.to(device)
        decoder_targets = decoder_targets.to(device)

        # 기울기 초기화
        optimizer.zero_grad()

        # 순방향 전파
        # outputs.shape == (batch_size, seq_len, tar_vocab_size)
        outputs = model(encoder_inputs, decoder_inputs)

        # 손실 계산 및 역방향 전파
        # outputs.view(-1, outputs.size(-1))의 shape는 (batch_size * seq_len, tar_vocab_size)
        # decoder_targets.view(-1)의 shape는 (batch_size * seq_len)
        loss = loss_function(outputs.view(-1, outputs.size(-1)), decoder_targets.view(-1))
        loss.backward()

        # 가중치 업데이트
        optimizer.step()

    train_loss, train_acc = evaluation(model, train_dataloader, loss_function, device)
    valid_loss, valid_acc = evaluation(model, valid_dataloader, loss_function, device)
    
    train_history['Loss'].append(train_loss)
    train_history['Acc'].append(train_acc)
    
    valid_history['Loss'].append(valid_loss)
    valid_history['Acc'].append(valid_acc)

    print(f'Epoch: {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f} | Valid Loss: {valid_loss:.4f} | Valid Acc: {valid_acc:.4f}')

    # 검증 손실이 최소일 때 체크포인트 저장
    if valid_loss < best_val_loss:
        print(f'Validation loss improved from {best_val_loss:.4f} to {valid_loss:.4f}. 체크포인트를 저장합니다.')
        best_val_loss = valid_loss
        torch.save(model.state_dict(), 'best_model_kkma.pth')


Epoch: 1/10 | Train Loss: 4.9691 | Train Acc: 0.2114 | Valid Loss: 5.0418 | Valid Acc: 0.2073
Validation loss improved from inf to 5.0418. 체크포인트를 저장합니다.
Epoch: 2/10 | Train Loss: 3.1123 | Train Acc: 0.5290 | Valid Loss: 3.2457 | Valid Acc: 0.5230
Validation loss improved from 5.0418 to 3.2457. 체크포인트를 저장합니다.
Epoch: 3/10 | Train Loss: 2.2093 | Train Acc: 0.6522 | Valid Loss: 2.3932 | Valid Acc: 0.6426
Validation loss improved from 3.2457 to 2.3932. 체크포인트를 저장합니다.
Epoch: 4/10 | Train Loss: 1.8046 | Train Acc: 0.6892 | Valid Loss: 2.0372 | Valid Acc: 0.6781
Validation loss improved from 2.3932 to 2.0372. 체크포인트를 저장합니다.
Epoch: 5/10 | Train Loss: 1.5763 | Train Acc: 0.7097 | Valid Loss: 1.8571 | Valid Acc: 0.6949
Validation loss improved from 2.0372 to 1.8571. 체크포인트를 저장합니다.
Epoch: 6/10 | Train Loss: 1.4063 | Train Acc: 0.7240 | Valid Loss: 1.7359 | Valid Acc: 0.7055
Validation loss improved from 1.8571 to 1.7359. 체크포인트를 저장합니다.
Epoch: 7/10 | Train Loss: 1.2323 | Train Acc: 0.7516 | Valid Loss: 

In [ ]:
# show plot

TH = len(train_history['Loss'])
plt.figure(figsize=(10, 4))
# Loss
plt.subplot(1, 2, 1)
plt.plot(range(1, TH+1), train_history['Loss'], 'r-', label='Train')
plt.plot(range(1, TH+1), valid_history['Loss'], 'b-', label='Valid')
plt.grid()
plt.title('Loss over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

# Accuracy
plt.subplot(1, 2, 2)
plt.plot(range(1, TH+1), train_history['Acc'], 'r-', label='Train')
plt.plot(range(1, TH+1), valid_history['Acc'], 'b-', label='Valid')
plt.grid()
plt.title('Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

#### 모델 로드

In [63]:
# 모델 로드
model.load_state_dict(torch.load('best_model_pron.pth'))

# 모델을 device에 올립니다.
model.to(device)

# 검증 데이터에 대한 정확도와 손실 계산
val_loss, val_accuracy = evaluation(model, valid_dataloader, loss_function, device)

print(f'Best model validation loss: {val_loss:.4f}')
print(f'Best model validation accuracy: {val_accuracy:.4f}')


Best model validation loss: 1.3893
Best model validation accuracy: 0.7585


In [65]:
print(tar_vocab['<sos>'])
print(tar_vocab['<eos>'])


2299
6791


#### seq2seq 번역기 동작시키기

In [66]:
index_to_src = {v: k for k, v in src_vocab.items()}
index_to_tar = {v: k for k, v in tar_vocab.items()}

# 원문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_src(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0):
      sentence = sentence + index_to_src[encoded_word] + ' '
  return sentence

# 번역문의 정수 시퀀스를 텍스트 시퀀스로 변환
def seq_to_tar(input_seq):
  sentence = ''
  for encoded_word in input_seq:
    if(encoded_word != 0 and encoded_word != tar_vocab['<sos>'] and encoded_word != tar_vocab['<eos>']):
      sentence = sentence + index_to_tar[encoded_word] + ' '
  return sentence


In [67]:
print(encoder_input_test[25])
print(decoder_input_test[25])
print(decoder_target_test[25])

[ 448 1003 1777 4902 4443  215  314 4173 2035 1545 4854    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0]
[2299  779 1785 3191  319 9079  382  550 7586 3662 2780 3950    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]
[ 779 1785 3191  319 9079  382  550 7586 3662 2780 3950 6791    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0]


In [71]:
def decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, max_output_len, int_to_src_token, int_to_tar_token):
    encoder_inputs = torch.tensor(input_seq, dtype=torch.long).unsqueeze(0).to(device)

    # 인코더의 초기 상태 설정
    hidden, cell = model.encoder(encoder_inputs)

    # 시작 토큰 <sos>을 디코더의 첫 입력으로 설정
    # unsqueeze(0)는 배치 차원을 추가하기 위함.
    decoder_input = torch.tensor([2299], dtype=torch.long).unsqueeze(0).to(device)

    decoded_tokens = []

    # for문을 도는 것 == 디코더의 각 시점
    for _ in range(max_output_len):
        output, hidden, cell = model.decoder(decoder_input, hidden, cell)

        # 소프트맥스 회귀를 수행. 예측 단어의 인덱스
        output_token = output.argmax(dim=-1).item()

        # 종료 토큰 <eos>
        if output_token == 6791:
            break

        # 각 시점의 단어(정수)는 decoded_tokens에 누적하였다가 최종 번역 시퀀스로 리턴합니다.
        decoded_tokens.append(output_token)

        # 현재 시점의 예측. 다음 시점의 입력으로 사용된다.
        decoder_input = torch.tensor([output_token], dtype=torch.long).unsqueeze(0).to(device)

    return ' '.join(int_to_tar_token[token] for token in decoded_tokens)


In [69]:
encoder_input_test[1]

array([3431, 3773, 3823, 2427,  874,  904, 3630, 1418, 2350, 2401, 1576,
       2818,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0])

In [72]:
#### 번역 모델 결과 확인
for seq_index in [3, 50, 100, 300, 1001]:
  input_seq = encoder_input_test[seq_index]
  translated_text = decode_sequence(input_seq, model, src_vocab_size, tar_vocab_size, 100, index_to_src, index_to_tar)

  print("입력문장 :",seq_to_src(encoder_input_test[seq_index]))
  print("정답문장 :",seq_to_tar(decoder_input_test[seq_index]))
  print("번역문장 :",translated_text)
  print("-"*50)


입력문장 : 산 에 가서 조금 만 다녀 보 만 지팡이 만들 좋은 나무 가 아주 많습니다 
정답문장 : 산 에 가서 조금 만 댕겨 보면 지 패 이 만들 좋은 나무 가 천지 삐 까리 데이 
번역문장 : 산 에 가서 조금 만 댕겨 보면 지 팽이 만들기 좋은 나무 가 천 지 삐 까리 시더
--------------------------------------------------
입력문장 : 남 의 무 구덩이 가 어디 인지 아는 사람 은 다른 집 무 를 빼 가 기도 했지 
정답문장 : 너 메 무시 구 디 가 어디 인지 아는 사람 은 다른 집 무시 를 빼 가 기도 해찌롱 
번역문장 : 너믜 무시 구 디 가 어딘 지 아는 사람 들 은 다른 집 무시 를 빼 가 기도 핻찌 로
--------------------------------------------------
입력문장 : 첨 시집 왔을 때 저 생선 국 고기 국 비린내 안 나나 어떻게 먹을까 했는데 막상 먹어 보니 괜찮네 
정답문장 : 첨 시집 왔을 때 저 생쌩국 고개 꾹 비렁내 안 나나 우 예 물까 했는데 마쌍 무거 보이 괜찮네 
번역문장 : 처음 시집 왔을 때 생선 국 을 보고 이 거 비렁내 가 나가 우 예 무글 지 걱정 핸 니 더
--------------------------------------------------
입력문장 : 올해 는 날씨 도 좋고 비도 적당하게 와서 밭 농사 정말 잘 되는 편이 야 
정답문장 : 올해 는 날씨 도 좋고 비도 적땅 하구로 와가 밭 농사 정말 잘 되는 펴니라야 
번역문장 : 올해 는 날씨 도 좋았고 비도 적당하구로 와가 밭농사 가 정말 잘 된 편이 다
--------------------------------------------------
입력문장 : 장작 으로 쓸 나무 는 쪼개서 저쪽 담장 밑 에 쌓아 두면 됩니다 
정답문장 : 둔가지로 쓸 나무 는 또 갈라가 쪼개 담장 밑 에 쌓아 두면 뎀니더 
번역문장 : 둥가지 로 쓸 나무 는 또 갈라가 저쪽 담장 밑 에 쌓아 두면 됩니